In [2]:
import os
import json
from moviepy import (
    AudioFileClip,
    CompositeVideoClip,
    ColorClip,
    TextClip,
    VideoFileClip,
    concatenate_videoclips,
    vfx
)
import re
import random

In [3]:
def demoji(text):
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [4]:
INPUT_FILE = r"./export/result.json"
INPUT_DIR = r"./export/"          # папка с исходными видео
OUTPUT_FILE = r"./result/output.mp4"    # итоговый файл
W, H = 1920, 1080
VIDEO_FONT_PATH = "./static/Roboto-Regular.ttf"
FADE_DURATION = 0.5   # плавный переход между видео

TITLE_FONT_PATH = "./static/Roboto-Bold.ttf"
TITLE_TEXT = "АНТИДОБРОКЕК #7"
INTRO_MUSIC = "./static/intro.mp3"         # музыка для вступления
TITLE_DURATION = 4.65  # секунды

In [5]:
export_dict = {}
with open(INPUT_FILE, 'r') as input_file:
    export_dict = json.load(input_file)

In [6]:
videos = []
messages = random.sample(export_dict["messages"], k=len(export_dict["messages"]))
for message in messages:
    if message.get("media_type", "unknown")!= "video_file":
        continue

    title = ""
    skip = False
    for text_entity in message["text_entities"]:
        if text_entity["type"] in ("hashtag") and text_entity["text"] == "#dobrokek":
            skip = True
            break
        elif text_entity["type"] not in ("plain", "code"):
            continue
        title += text_entity["text"].replace("\n\n", " ").strip()+" "
    if skip:
        continue
    video = {
        "filepath": os.path.join(INPUT_DIR, message["file"]),
        "title": demoji(title).strip(),
    }

    videos.append(video)


In [7]:
title_bg = ColorClip((W, H), color=(0, 0, 0), duration=TITLE_DURATION)

# Текст
title_text = TextClip(
    font=TITLE_FONT_PATH,
    text=TITLE_TEXT,
    color="white",
    font_size=60,
    size=(W, int(H * 0.8)),
    text_align="center",
).with_duration(TITLE_DURATION).with_position(("center", "center"))

# Собираем титр
title_clip = CompositeVideoClip([title_bg, title_text])

# Музыка на титры
audio_tracks = []
if os.path.exists(INTRO_MUSIC):
    print("Intro music added")
    intro_music = AudioFileClip(INTRO_MUSIC).with_volume_scaled(0.2).subclipped(0, TITLE_DURATION)
    title_clip = title_clip.with_audio(intro_music)
    audio_tracks.append(intro_music)


Intro music added


In [8]:
video_tracks = [title_clip.with_start(0)]
current_time = TITLE_DURATION

In [9]:
clips = [title_clip]

In [9]:
FIRST_CLIP_PATH = r"./export/first_clip.MP4"

In [ ]:
first_clip = VideoFileClip(FIRST_CLIP_PATH)

# Масштабирование
first_clip_resized = first_clip.resized(height=H)
if first_clip_resized.w > W:
    first_clip_resized = first_clip_resized.resized(width=W)

# Подложка
first_clip_bg = ColorClip((W, H), color=(0, 0, 0), duration=first_clip.duration)


# Собираем видео
first_clip_composite = CompositeVideoClip([
    first_clip_bg,
    first_clip_resized.with_position("center"),
]).with_audio(first_clip.audio)


clips.append(first_clip_composite)

In [10]:
for video in videos:
    try:
        clip = VideoFileClip(video["filepath"])

        # Масштабирование
        clip_resized = clip.resized(height=H)
        if clip_resized.w > W:
            clip_resized = clip_resized.resized(width=W)

        # Подложка
        bg = ColorClip((W, H), color=(0, 0, 0), duration=clip.duration)

        # Подпись к видео — в углу справа
        caption_w = (W-clip_resized.w)//2
        caption = TextClip(
            font=VIDEO_FONT_PATH,
            size=(caption_w, H),
            text=video["title"],
            color="white",
            font_size=20,
        ).with_duration(clip.duration).with_position(("right", "top"))

        # Собираем видео
        composite = CompositeVideoClip([
            bg,
            clip_resized.with_position("center"),
            caption
        ]).with_audio(clip.audio)


        clips.append(composite)
    except:
        continue

In [14]:
LAST_CLIP_PATH = r"./export/last_clip.mp4"

In [15]:
last_clip = VideoFileClip(LAST_CLIP_PATH)

# Масштабирование
last_clip_resized = last_clip.resized(height=H)
if last_clip_resized.w > W:
    last_clip_resized = last_clip_resized.resized(width=W)

# Подложка
last_clip_bg = ColorClip((W, H), color=(0, 0, 0), duration=last_clip.duration)


# Собираем видео
last_clip_composite = CompositeVideoClip([
    last_clip_bg,
    last_clip_resized.with_position("center"),
]).with_audio(last_clip.audio)


clips.append(last_clip_composite)

In [11]:
clips.append(title_clip)

In [12]:
def make_transition(duration, w=1920, h=1080):
    return (
        ColorClip((w, h), color=(0, 0, 0), duration=duration)
        .with_effects([vfx.FadeIn(duration/2,0), vfx.FadeOut(duration/2,0)])
    )

transition = make_transition(FADE_DURATION)

In [13]:
final = concatenate_videoclips(
    clips,
    method="chain",
    transition=transition
)

final.write_videofile(
    OUTPUT_FILE,
    fps=30,
    threads=12,
)

MoviePy - Building video ./result/output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ./result/output.mp4



frame_index:  19%|█▉        | 3158/16635 [06:41<29:12,  7.69it/s, now=None]c:\Users\AlexS\miniconda3\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:190: UserWarning: In file ./export/video_files/reels_20260125_191328.mp4, 2764800 bytes wanted but 0 bytes read at frame index 3002 (out of a total 3005 frames), at time 100.07/100.17 sec. Using the last valid frame instead.
  warnings.warn(
frame_index:  19%|█▉        | 3159/16635 [06:41<29:12,  7.69it/s, now=None]c:\Users\AlexS\miniconda3\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:190: UserWarning: In file ./export/video_files/reels_20260125_191328.mp4, 2764800 bytes wanted but 0 bytes read at frame index 3003 (out of a total 3005 frames), at time 100.10/100.17 sec. Using the last valid frame instead.
  warnings.warn(
frame_index:  19%|█▉        | 3160/16635 [06:41<29:26,  7.63it/s, now=None]c:\Users\AlexS\miniconda3\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:190: UserWarning: In file ./export/video_files/reels_2026

MoviePy - Done !
MoviePy - video ready ./result/output.mp4
